In [1]:
from qoilsimul import *

# 1. Pre-compensation

In [104]:
expr = Experiment()

alpha, beta, gamma, delta = sp.var('α β γ δ')

c = Photon(pathModes=1,dof=['path','pol'],state = alpha*co(p1,H) + beta*co(p1,V))
t = Photon(pathModes=1,dof=['path','pol'],state = gamma*co(p1,H) + delta*co(p1,V))
expr.addPhotons(c,t)

hwp1 = HWP(th=sp.pi/8)
ppbsh1 = PPBSH(r=sp.Rational(2)/3)
ppbsh2 = PPBSH(r=sp.Rational(2)/3)
ppbsv = PPBSV(r=sp.Rational(2)/3)
m1 = PBS()
m2 = PBS()

hwp2 = HWP(th=sp.pi/8)

hwp1.i = [t.o[0]]
ppbsh1.i = [c.o[0], expr.emptyMode()]
ppbsh2.i = [hwp1.o[0], expr.emptyMode()]
ppbsv.i = [ppbsh1.o[0],ppbsh2.o[0]]
hwp2.i = [ppbsv.o[1]]
# m1.i = [ppbsv.o[0], expr.emptyMode()] # H, V
# m2.i = [ppbsv.o[1], expr.emptyMode()] # H, V


expr.addElements(hwp1,ppbsh1,ppbsh2,ppbsv,hwp2)#,m1,m2)

d = Detectors(numberOfDetectors=2)
d.i = [ppbsv.o[0], hwp2.o[0]]
# d.i = [m1.o[1], m2.o[0]]

expr.addDetectors(d)

expr.generateCircuit()
print(expr.circuit)
expr.state.expand()

Experiment initialised.
a : p -----PPBSH-------PPBSV-------D
b : p -HWP-------PPBSH-PPBSV-HWP---D
c : .......PPBSH--------------------
d : .............PPBSH--------------



α*γ*x[a, H]*x[b, H] + α*δ*x[a, H]*x[b, V] + β*γ*x[a, V]*x[b, H] + β*δ*x[a, V]*x[b, V]

In [105]:
expr.build()
print(expr.successProbability)
expr.postSelectedState

α**2*γ**2/9 + α**2*δ**2/9 + β**2*γ**2/9 + β**2*δ**2/9


α*γ*x[a, H]*x[b, H]/3 + α*δ*x[a, H]*x[b, V]/3 + β*γ*x[a, V]*x[b, V]/3 + β*δ*x[a, V]*x[b, H]/3

In [120]:
expr.postSelectedState.subs({alpha: 0,  # H
                             beta:  1,  # V
                             gamma: 1,  # H
                             delta: 0}) # V

x[a, V]*x[b, V]/3

# 2. Post-compensation 

In [114]:
expr = Experiment()

c = Photon(pathModes=1,dof=['path','pol'],state = alpha*co(p1,H) + beta*co(p1,V))
t = Photon(pathModes=1,dof=['path','pol'],state = gamma*co(p1,H) + delta*co(p1,V))
expr.addPhotons(c,t)

hwp1 = HWP(th=sp.pi/8)
ppbsh1 = PPBSH(r=sp.Rational(2)/3)
ppbsh2 = PPBSH(r=sp.Rational(2)/3)
ppbsv = PPBSV(r=sp.Rational(2)/3)
hwp2 = HWP(th=sp.pi/8)

hwp1.i = [t.o[0]]
ppbsv.i = [c.o[0], hwp1.o[0]]
ppbsh1.i = [ppbsv.o[0], expr.emptyMode()]
ppbsh2.i = [ppbsv.o[1], expr.emptyMode()]
hwp2.i = [ppbsh2.o[0]]

expr.addElements(hwp1,ppbsv,ppbsh1,ppbsh2,hwp2)

d = Detectors(numberOfDetectors=2)
d.i = [ppbsh1.o[0],ppbsh2.o[0]]

expr.addDetectors(d)

Experiment initialised.


In [115]:
expr.generateCircuit()
print(expr.circuit)

a : p -----PPBSV-PPBSH-------------D
b : p -HWP-PPBSV-------PPBSH-HWP---D
c : .............PPBSH--------------
d : ...................PPBSH--------



In [116]:
expr.build()
expr.postSelectedState

α*γ*x[a, H]*x[b, H]/3 + α*δ*x[a, H]*x[b, V]/3 + β*γ*x[a, V]*x[b, V]/3 + β*δ*x[a, V]*x[b, H]/3

In [119]:
expr.postSelectedState.subs({alpha: 0,  # H
                             beta:  1,  # V
                             gamma: 1,  # H
                             delta: 0}) # V

x[a, V]*x[b, V]/3